> title : 200_etri_lifelog_model_vF (모델 학습 코드) <br>
 -  코드 실행 전 PATH 변경하세요.
  - PATH  =  '/content/drive/MyDrive/data/ch2025_data_items/share/submissions/

### 🔨 PATH 설정

In [1]:
PATH  =  '/content/drive/MyDrive/data/ch2025_data_items/share/submissions/input' ### <---- 코드 실행 전 PATH 변경하세요.

In [2]:
# 데이터는 구글드라이브에 저장되어 있어서 구글드라이브 마운트를 합니다.
# 데이터 저장 PATH를 변경하시면 아래 구글드라이브 마운트를 주석처리하시면 됩니다.
from google.colab import drive, files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 📚 라이브러리

In [3]:
# 라이브러리 설치
# ! pip install haversine >/dev/null
# ! pip install optuna  >/dev/null
# ! pip install category_encoders >/dev/null
# ! pip install tabpfn  >/dev/null
# ! pip install catboost >/dev/null
# ! pip install torchmetrics >/dev/null

In [4]:
# 기본 모듈
import os
import sys
import re
import ast
import glob
import random
import warnings
from collections import Counter
from math import radians, cos, sin, asin, sqrt
from functools import reduce
from datetime import datetime, timedelta, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 머신러닝
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import f1_score, roc_auc_score, roc_curve
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
from category_encoders import TargetEncoder
from lightgbm import LGBMClassifier, log_evaluation, early_stopping
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import lightgbm as lgb
from tabpfn import TabPFNClassifier
from sklearn.inspection import permutation_importance
import shap

# PyTorch
import torch
from torch import nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Hugging Face
from huggingface_hub import login
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    LlamaTokenizer,
    LlamaForCausalLM,
    LlamaForSequenceClassification
)

# PEFT (Parameter-Efficient Fine-Tuning)
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    TaskType
)

# Evaluation & Utilities
from torchmetrics import Accuracy

# 기타
from tqdm import tqdm
from tqdm.auto import tqdm as auto_tqdm  # 필요 시 구분
from scipy.stats import entropy
from haversine import haversine
from io import StringIO
import gc

# wandb
# import wandb
# wandb.login(key="5fa8dfb2c5be3c888bfe0101437a8fa22fbdf0e0")
# wandb.init(project="etri_lifelog", entity="byc3230")

# 옵션
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%0.4f' % x)

# 기타
warnings.filterwarnings('ignore')

# 기본 시드 설정
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# 재현성 향상 옵션 (단, 성능 저하 가능성 있음)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [5]:
def run_basemodel(train, test, valid_ids, common_params, n_splits, n_repeats, random_state=42, AutoTab=False, S1_balance=True):

    lgb_A = 0.333
    xgb_B = 0.334
    tab_C = 0.333

    lgb_params = common_params['Q1'].copy()
    lgb_params['random_state'] = random_state

    train_df = train.copy()
    test_df = test.copy()

    submission_final = test_df[['subject_id', 'sleep_date', 'lifelog_date']].copy()
    submission_final['lifelog_date'] = pd.to_datetime(submission_final['lifelog_date']).dt.date

    # 타겟
    targets_binary = ['Q1', 'Q2', 'Q3', 'S2', 'S3']
    targets_binary_name = ['기상직후수면질','취침전신체적피로','취침전스트레스','수면효율','수면잠들기시간']
    target_multiclass = 'S1'
    all_targets = targets_binary + [target_multiclass]

    # 노이즈 수준 설정
    def add_noise(series, noise_level, seed=3):
        rng = np.random.default_rng(seed)
        return series * (1 + noise_level * rng.standard_normal(len(series)))

    noise_level = 0.015  # 필요에 따라 조정

    # ======================================================================================

    # predweekday 생성: subject_id별로 sleep_duration_min이 긴 요일 2개를 선정
    top2_days = (
        train_df.groupby(['subject_id', 'weekday'])['sleep_duration_min']
        .mean()
        .reset_index()
        .sort_values(['subject_id', 'sleep_duration_min'], ascending=[True, False])
        .groupby('subject_id')
        .head(1)
    )

    # subject_id별 top2 weekday 집합 만들기
    top2_day_dict = top2_days.groupby('subject_id')['weekday'].apply(set).to_dict()

    # train_df, test_df에 predweekday 컬럼 추가
    def mark_predweekday(row):
        return int(row['weekday'] in top2_day_dict.get(row['subject_id'], set()))

    train_df['predweekday'] = train_df.apply(mark_predweekday, axis=1)
    test_df['predweekday'] = test_df.apply(mark_predweekday, axis=1)

    # ======================================================================================

    # [1]
    train_df['new1'] = np.where(train_df['img0']>0,1,0)
    train_df['new2'] = np.where(train_df['light_wake_time_diff']>0,1,0)
    train_df['new3'] = np.where(train_df['light_sleep_time_ratio']>0,1,0)
    train_df['new4'] = np.where(train_df['light_wake_time_ratio']>0,1,0)
    train_df['new5'] = np.where(train_df['light_sleep_duration_ratio']>0,1,0)
    train_df['new6'] = np.where(train_df['sleep_duration_vs_weekday_avg']>0,1,0) # 1.7583333333333333
    train_df['new7'] = np.where(train_df['wake_time_ratio']>1,1,0)               # 1.7583333333333333
    train_df['new8'] = np.where(train_df['sleep_duration_ratio']>1,1,0)          # 1.7583333333333333

    # [2]
    test_df['new1'] = np.where(test_df['img0']>0,1,0)
    test_df['new2'] = np.where(test_df['light_wake_time_diff']>0,1,0)
    test_df['new3'] = np.where(test_df['light_sleep_time_ratio']>0,1,0)
    test_df['new4'] = np.where(test_df['light_wake_time_ratio']>0,1,0)
    test_df['new5'] = np.where(test_df['light_sleep_duration_ratio']>0,1,0)
    test_df['new6'] = np.where(test_df['sleep_duration_vs_weekday_avg']>0,1,0)   # 1.7583333333333333
    test_df['new7'] = np.where(test_df['wake_time_ratio']>1,1,0)                 # 1.7583333333333333
    test_df['new8'] = np.where(test_df['sleep_duration_ratio']>1,1,0)            # 1.7583333333333333

    # ======================================================================================

    # 타겟인코딩
    for new in ['new1','new2','new6','new7','new8','weekend']:

      for tgt in all_targets:

        encoder_feats = ['subject_id','month',new] # 'weekday', 'subject_id','month','weekend'

        #### 타겟인코딩1

        subject_mean = train_df.groupby(encoder_feats)[tgt].mean().rename(f'{tgt}_{encoder_feats[2]}_te')
        train_df = train_df.merge(subject_mean, on=encoder_feats, how='left')
        test_df = test_df.merge(subject_mean, on=encoder_feats, how='left')
        global_mean = train_df[tgt].mean()
        test_df[f'{tgt}_{encoder_feats[2]}_te'] = test_df[f'{tgt}_{encoder_feats[2]}_te'].fillna(global_mean)

        # 노이즈 추가
        train_df[f'{tgt}_{encoder_feats[2]}_te'] = add_noise(train_df[f'{tgt}_{encoder_feats[2]}_te'], noise_level)
        test_df[f'{tgt}_{encoder_feats[2]}_te'] = add_noise(test_df[f'{tgt}_{encoder_feats[2]}_te'], noise_level)

        #### 타겟인코딩2

        # 새로운 범주형 열 생성
        train_df['TMP'] = train_df[encoder_feats].applymap(str).apply(lambda x: ''.join(x) ,axis=1)
        test_df['TMP'] = test_df[encoder_feats].applymap(str).apply(lambda x: ''.join(x) ,axis=1)

        # 인코더
        encoder = TargetEncoder(cols=['TMP'], smoothing=300) # 40
        encoder.fit(train_df[['TMP']], train_df[tgt])

        # 인코딩 결과를 새로운 열에 저장
        train_df[f'{tgt}_{encoder_feats[2]}_te2'] = encoder.transform(train_df[['TMP']])
        test_df[f'{tgt}_{encoder_feats[2]}_te2'] = encoder.transform(test_df[['TMP']])

        # 노이즈 추가
        train_df[f'{tgt}_{encoder_feats[2]}_te2'] = add_noise(train_df[f'{tgt}_{encoder_feats[2]}_te2'], noise_level)
        test_df[f'{tgt}_{encoder_feats[2]}_te2'] = add_noise(test_df[f'{tgt}_{encoder_feats[2]}_te2'], noise_level)

        # 불필요한 변수 제거
        train_df = train_df.drop(columns=['TMP'])
        test_df = test_df.drop(columns=['TMP'])


    # 인코딩
    PK = ['sleep_date', 'lifelog_date', 'subject_id']
    encoder = LabelEncoder()
    categorical_features = [i for i in train_df.select_dtypes(include=['object', 'category']).columns if i not in PK+['pk']]
    for col in categorical_features:
        print(col)
        train_df[col] = encoder.fit_transform(train_df[col])
        test_df[col] = encoder.fit_transform(test_df[col])

    # X
    X = train_df.drop(columns=PK + all_targets)
    test_X = test_df.drop(columns=PK + all_targets)
    print(f'# X shape: {X.shape}')
    print(f'# test_X shape: {test_X.shape}')
    print(f'# valid_ids shape: {valid_ids.shape}')

    # =============================================================================================================================
    # valid  valid  valid  valid  valid  valid  valid  valid  valid  valid  valid  valid  valid  valid  valid  valid  valid  valid
    # =============================================================================================================================

    print('\n STEP1: 실험 결과 확인')
    print("=============== Validation Results ==============")
    total_avg_f1s = []
    val_f1 = []
    binary_val_preds = {}
    multiclass_val_preds = {}
    binary_test_preds = {}
    multiclass_test_preds = {}
    test_preds = {}
    xfeatures_dict = {}

    # Find optimal weights
    best_weights = []
    best_scores = []

    # ------
    # binary
    # ------

    for col in targets_binary:

        y = train_df[col]

        valid_ids['pk'] = valid_ids['subject_id']+valid_ids['sleep_date']
        train_df['pk'] = train_df['subject_id']+train_df['sleep_date']

        X_valid = train_df.loc[train_df['pk'].isin(valid_ids['pk']),X.columns.tolist()].reset_index(drop=True).copy()
        X_train = train_df.loc[~train_df['pk'].isin(valid_ids['pk']),X.columns.tolist()].reset_index(drop=True).copy()
        y_valid = train_df.loc[train_df['pk'].isin(valid_ids['pk']),y.name].reset_index(drop=True).copy()
        y_train = train_df.loc[~train_df['pk'].isin(valid_ids['pk']),y.name].reset_index(drop=True).copy()

        # ---------------------------------- permutation_importance -----------------------------------------------

        feature_names = X.columns.tolist()

        if col in ['Q3']:
          # xfeatures1
          model = XGBClassifier(**xgb_params)
          model.fit(X_train, y_train)
          explainer = shap.TreeExplainer(model)
          shap_values = explainer.shap_values(X_train)
          shap_importance = np.abs(shap_values).mean(axis=0)
          shap_df = pd.DataFrame({
              'feature': X_train.columns,
              'shap_importance': shap_importance
          }).sort_values(by='shap_importance', ascending=False)
          xfeatures1 = shap_df.head(15)['feature'].tolist()
          print(f'# {col} of xfeatures1: {len(xfeatures1)}')
          print(f'# xfeatures1: {xfeatures1}')
        else:
          xfeatures1 = []

        # xfeatures2
        correlations = X.select_dtypes(include=['number']).corrwith(y)
        sorted_correlations = correlations.abs().sort_values(ascending=False)
        xfeatures2 = sorted_correlations[sorted_correlations>0.1].index.tolist()
        print(f'# {col} of xfeatures2: {len(xfeatures2)}')
        print(f'# xfeatures2: {xfeatures2}')
        print(f'=> {col} of xfeatures: {len(set(xfeatures1+xfeatures2))}')
        print('')

        xfeatures_dict[col] = [i for i in X.columns.tolist() if i in set(xfeatures1+xfeatures2)]

        # ---------------------------------------------------------------------------------

        X_valid = train_df.loc[train_df['pk'].isin(valid_ids['pk']),xfeatures_dict[col]].reset_index(drop=True).copy()
        X_train = train_df.loc[~train_df['pk'].isin(valid_ids['pk']),xfeatures_dict[col]].reset_index(drop=True).copy()
        y_valid = train_df.loc[train_df['pk'].isin(valid_ids['pk']),y.name].reset_index(drop=True).copy()
        y_train = train_df.loc[~train_df['pk'].isin(valid_ids['pk']),y.name].reset_index(drop=True).copy()

        # Train LightGBM
        lgb_model = LGBMClassifier(**lgb_params)
        lgb_model.fit(X_train, y_train)

        # Train XGBoost
        xgb_model = XGBClassifier(**xgb_params)
        xgb_model.fit(X_train, y_train)

        # Train TabPFN
        if AutoTab==False:
          tabpfn_model = TabPFNClassifier(**tabpfn_params)
        else:
          tabpfn_model = AutoTabPFNClassifier(
              device='cuda',
              # preset= 'avoid_overfitting',
              ges_scoring_string = 'f1',
              random_state = 42,
              max_time = 60*10                # 10분
          )
        tabpfn_model.fit(X_train, y_train)
        tab_pred_valid = tabpfn_model.predict_proba(X_valid.values)[:, 1]

        lgb_pred_valid = lgb_model.predict_proba(X_valid)[:, 1]
        xgb_pred_valid = xgb_model.predict_proba(X_valid)[:, 1]

        pred_valid = (lgb_A * lgb_pred_valid + xgb_B * xgb_pred_valid + tab_C * tab_pred_valid > 0.5).astype(int)

        f1 = f1_score(y_valid, pred_valid, average='macro')
        val_f1.append(f1)

        # Store predictions
        binary_val_preds[col] = {
            'lgb': lgb_pred_valid,
            'xgb': xgb_pred_valid,
            'tab': tab_pred_valid,
            'true': y_valid
        }


    # ----------
    # multiclass
    # ----------

    y = train_df[target_multiclass]
    X_valid = train_df.loc[train_df['pk'].isin(valid_ids['pk']),X.columns.tolist()].reset_index(drop=True).copy()
    X_train = train_df.loc[~train_df['pk'].isin(valid_ids['pk']),X.columns.tolist()].reset_index(drop=True).copy()
    y_valid = train_df.loc[train_df['pk'].isin(valid_ids['pk']),y.name].reset_index(drop=True).copy()
    y_train = train_df.loc[~train_df['pk'].isin(valid_ids['pk']),y.name].reset_index(drop=True).copy()

    # ---------------------------------- permutation_importance -----------------------------------------------

    feature_names = X.columns.tolist()
    xfeatures1 = []
    correlations = X.select_dtypes(include=['number']).corrwith(y)
    sorted_correlations = correlations.abs().sort_values(ascending=False)
    xfeatures2 = sorted_correlations[sorted_correlations>0.1].index.tolist()
    print(f'# S1 of xfeatures2: {len(xfeatures2)}')
    print(f'# xfeatures2: {xfeatures2}')
    print(f'=> S1 of xfeatures: {len(set(xfeatures1+xfeatures2))}')
    print('')

    xfeatures_dict['S1'] = [i for i in X.columns.tolist() if i in set(xfeatures1+xfeatures2)]

    # ----------------------------------------------------------------------------------------------------------


    y = train_df[target_multiclass]
    X_valid = train_df.loc[train_df['pk'].isin(valid_ids['pk']),xfeatures_dict['S1']].reset_index(drop=True).copy()
    X_train = train_df.loc[~train_df['pk'].isin(valid_ids['pk']),xfeatures_dict['S1']].reset_index(drop=True).copy()
    y_valid = train_df.loc[train_df['pk'].isin(valid_ids['pk']),y.name].reset_index(drop=True).copy()
    y_train = train_df.loc[~train_df['pk'].isin(valid_ids['pk']),y.name].reset_index(drop=True).copy()

    # 클래스 weight 계산
    classes = np.unique(y_train)
    weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
    class_weights = dict(zip(classes, weights))

    # 각 샘플에 대해 weight 매핑
    w_train = pd.Series(y_train).map(class_weights)
    w_train = compute_sample_weight(class_weight='balanced', y=y_train)

    if S1_balance==True:
      # Train LightGBM
      lgb_model = LGBMClassifier(**lgb_params, objective='multiclass', num_class=3)
      lgb_model.fit(X_train, y_train, sample_weight=w_train)

      # Train XGBoost
      xgb_model = XGBClassifier(**xgb_params_S1, objective='multi:softmax', num_class=3)  ############## check ###############
      xgb_model.fit(X_train, y_train,sample_weight=w_train)
    else:
      # Train LightGBM
      lgb_model = LGBMClassifier(**lgb_params, objective='multiclass', num_class=3)
      lgb_model.fit(X_train, y_train)

      # Train XGBoost
      xgb_model = XGBClassifier(**xgb_params_S1, objective='multi:softmax', num_class=3)  ############## check ###############
      xgb_model.fit(X_train, y_train)

    # Train TabPFN
    if AutoTab==False:
      tabpfn_model = TabPFNClassifier(**tabpfn_params)
    else:
      tabpfn_model = AutoTabPFNClassifier(
          device='cuda',
          # preset= 'avoid_overfitting',
          ges_scoring_string = 'f1',
          random_state = 42,
          max_time = 60*10                # 10분
      )
    tabpfn_model.fit(X_train, y_train)

    # Get predictions and ensemble
    lgb_pred_valid = lgb_model.predict_proba(X_valid)
    xgb_pred_valid = xgb_model.predict_proba(X_valid)
    tab_pred_valid = tabpfn_model.predict_proba(X_valid.values)

    pred_valid = np.argmax(lgb_A * lgb_pred_valid + xgb_B * xgb_pred_valid + tab_C * tab_pred_valid, axis=1)

    f1 = f1_score(y_valid, pred_valid, average='macro')
    val_f1.append(f1)

    multiclass_val_preds = {
        'lgb': lgb_pred_valid,
        'xgb': xgb_pred_valid,
        'tab': tab_pred_valid,
        'true': y_valid
    }


    # ---------------------------------------- Generate all possible weight combinations that sum to 1 ----------------------------------------

    from itertools import product

    step = 0.1
    candidates = np.arange(0, 1.1, step)

    for lgb_A, xgb_B, tab_C in product(candidates, repeat=3):
        total = lgb_A + xgb_B + tab_C
        if np.isclose(total, 1.0):
            weights = (lgb_A, xgb_B, tab_C)
            val_scores = []

            # Binary targets
            for col in targets_binary:
                preds = binary_val_preds[col]
                blended = lgb_A * preds['lgb'] + xgb_B * preds['xgb'] + tab_C * preds['tab']
                val_scores.append(f1_score(preds['true'], (blended > 0.5).astype(int), average='macro'))

            # Multiclass target
            preds = multiclass_val_preds
            blended = lgb_A * preds['lgb'] + xgb_B * preds['xgb'] + tab_C * preds['tab']
            val_scores.append(f1_score(preds['true'], np.argmax(blended, axis=1), average='macro'))

            best_weights.append(weights)
            best_scores.append(np.mean(val_scores))

    # ------------------------------------------------------------------------------------------------------------------------------------------

    # Sort results and get top
    sorted_indices = np.argsort(best_scores)[::-1]
    top_weights = [best_weights[i] for i in sorted_indices]
    top_scores = [best_scores[i] for i in sorted_indices]

    # eval
    avg_f1 = np.mean(val_f1)
    total_avg_f1s.append(avg_f1)
    detail = " ".join([f"{name}({tname}):{score:.4f}" for name, tname, score in zip(targets_binary + [target_multiclass], targets_binary_name + ['S1'], val_f1)])
    print(f" 평균 F1: {avg_f1:.4f} / [상세] {detail}")
    print(f"# 전체 평균 F1: {np.mean(total_avg_f1s):.4f}")
    print("================================================")

    # ============================================================================================================================================
    # modoling with 100% train & no valid  modoling with 100% train & no valid  modoling with 100% train & no valid  modoling with 100% train & no
    # ============================================================================================================================================

    print('\n STEP2: 전체 데이터로 모델 재학습')
    print("====== modeling with 100% train & no valid =====")

    # binary
    binary_preds = {}
    binary_preds_proba = {}
    for col in targets_binary:

        y = train_df[col]
        is_multiclass = False

        # Train LightGBM
        lgb_model = LGBMClassifier(**lgb_params)
        lgb_model.fit(X[xfeatures_dict[col]], y)

        # Train XGBoost
        xgb_model = XGBClassifier(**xgb_params)
        xgb_model.fit(X[xfeatures_dict[col]], y)

        # Train TabPFN
        if AutoTab==False:
          tabpfn_model = TabPFNClassifier(**tabpfn_params)
        else:
          tabpfn_model = AutoTabPFNClassifier(
              device='cuda',
              # preset= 'avoid_overfitting',
              ges_scoring_string = 'f1',
              random_state = 42,
              max_time = 60*10                # 10분
          )
        tabpfn_model.fit(X[xfeatures_dict[col]], y)

        tab_pred = tabpfn_model.predict_proba(test_X[xfeatures_dict[col]])[:, 1]
        lgb_pred = lgb_model.predict_proba(test_X[xfeatures_dict[col]])[:, 1]
        xgb_pred = xgb_model.predict_proba(test_X[xfeatures_dict[col]])[:, 1]

        binary_preds[col] = (lgb_A * lgb_pred + xgb_B * xgb_pred + tab_C * tab_pred > 0.5).astype(int)

        # Store predictions
        binary_test_preds[col] = {
            'lgb': lgb_pred,
            'xgb': xgb_pred,
            'tab': tab_pred
        }

        # Feature importance (using LightGBM's importance)
        # fi_df = pd.DataFrame({'feature': X.columns, 'importance': lgb_model.feature_importances_})
        # top10 = fi_df.sort_values(by='importance', ascending=False).head(10)
        # feat_str = ", ".join([f"{row['feature']}({int(row['importance'])})" for _, row in top10.iterrows()])
        # print(f"[{col}] {feat_str}")


    # ----------
    # multiclass
    # ----------

    y = train_df['S1']

    # 클래스 weight 계산
    classes = np.unique(y)
    weights = compute_class_weight(class_weight='balanced', classes=classes, y=y)
    class_weights = dict(zip(classes, weights))

    # 각 샘플에 대해 weight 매핑
    w_train = pd.Series(y).map(class_weights)
    w_train = compute_sample_weight(class_weight='balanced', y=y)

    is_multiclass = True

    if S1_balance==True:
      # Train LightGBM
      lgb_model = LGBMClassifier(**lgb_params, objective='multiclass', num_class=3)
      lgb_model.fit(X[xfeatures_dict['S1']], y, sample_weight=w_train)

      # Train XGBoost
      xgb_model = XGBClassifier(**xgb_params_S1, objective='multi:softmax', num_class=3)  ############## check ###############
      xgb_model.fit(X[xfeatures_dict['S1']], y,sample_weight=w_train)
    else:
      # Train LightGBM
      lgb_model = LGBMClassifier(**lgb_params, objective='multiclass', num_class=3)
      lgb_model.fit(X[xfeatures_dict['S1']], y)

      # Train XGBoost
      xgb_model = XGBClassifier(**xgb_params_S1, objective='multi:softmax', num_class=3)  ############## check ###############
      xgb_model.fit(X[xfeatures_dict['S1']], y)

    # Train TabPFN
    if AutoTab==False:
      tabpfn_model = TabPFNClassifier(**tabpfn_params)
    else:
      tabpfn_model = AutoTabPFNClassifier(
          device='cuda',
          # preset= 'avoid_overfitting',
          ges_scoring_string = 'f1',
          random_state = 42,
          max_time = 60*10                # 10분
      )
    tabpfn_model.fit(X[xfeatures_dict['S1']], y)

    # Get predictions and ensemble
    lgb_pred = lgb_model.predict_proba(test_X[xfeatures_dict['S1']])
    xgb_pred = xgb_model.predict_proba(test_X[xfeatures_dict['S1']])
    tab_pred = tabpfn_model.predict_proba(test_X[xfeatures_dict['S1']])

    multiclass_test_preds = {
        'lgb': lgb_pred,
        'xgb': xgb_pred,
        'tab': tab_pred
    }

    multiclass_pred = np.argmax(lgb_A * lgb_pred + xgb_B * xgb_pred + tab_C * tab_pred, axis=1)
    multiclass_pred_proba = lgb_A * lgb_pred + xgb_B * xgb_pred + tab_C * tab_pred

    # Feature importance
    # fi_df = pd.DataFrame({'feature': X.columns, 'importance': lgb_model.feature_importances_})
    # top10 = fi_df.sort_values(by='importance', ascending=False).head(10)
    # feat_str = ", ".join([f"{row['feature']}({int(row['importance'])})" for _, row in top10.iterrows()])
    # print(f"[S1] {feat_str}")

    # 예측 저장
    submission_final['S1'] = multiclass_pred
    for col in targets_binary:
      submission_final[col] = binary_preds[col]
    submission_final = submission_final[['subject_id', 'sleep_date', 'lifelog_date', 'Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']]
    fname = f"submission_{np.mean(total_avg_f1s)}.csv"
    submission_final.to_csv(fname, index=False)
    print(f"# {fname} 저장 완료")
    print(f"# submission shape:{submission_final.shape}")
    print("================================================")

    # ---------------------------------------- Top 10 Weight Combinations ----------------------------------------

    submission_final_dict = {}
    print("\nTop 10 Weight Combinations:")
    for i, (weights, score) in enumerate(zip(top_weights[:10], top_scores[:10])):

        print(f"Rank {i+1}: lgb_A={weights[0]:.1f}, xgb_B={weights[1]:.1f}, tab_C={weights[2]:.1f} - Score: {score:.4f}")
        lgb_A, xgb_B, tab_C = weights

        # Binary predictions
        for col in targets_binary:
            preds = binary_test_preds[col]
            ensemble_pred = (lgb_A * preds['lgb'] + xgb_B * preds['xgb'] + tab_C * preds['tab'] > 0.5).astype(int)
            submission_final[col] = ensemble_pred

        # Multiclass prediction
        preds = multiclass_test_preds
        ensemble_pred = np.argmax(lgb_A * preds['lgb'] + xgb_B * preds['xgb'] + tab_C * preds['tab'], axis=1)
        submission_final['S1'] = ensemble_pred

        # 저장
        submission_final_dict[i] = submission_final.copy()
        fname = f"submission_top{i+1}_{score:.4f}.csv"
        submission_final_dict[i].to_csv(fname, index=False)
        print(f"Saved submission to {fname}")


    # 모델별 예측결과 비율 비교
    a11 = train_df[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].sum()
    a13 = train_df[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].apply(len)
    a12 = train_df[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].mean()
    a21 = submission_final_dict[0][['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].sum()
    a23 = submission_final_dict[0][['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].apply(len)
    a22 = submission_final_dict[0][['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']].mean()
    result = pd.concat([a11, a13, a12, a21, a23, a22], axis=1)
    result.columns = ['학습sum','학습len','학습mean','테스트sum','테스트len','테스트mean']
    print('\n STEP3: 예측결과 비교표')
    display(result)

    # S1분포
    a1 = train['S1'].value_counts(normalize=True)
    a2 = submission_final['S1'].value_counts(normalize=True)
    S1분포 = pd.concat([a1,a2],axis=1)
    display(S1분포)

    oof_result = []

    return submission_final_dict[0], oof_result

In [6]:
string = """
subject_id,sleep_date
id01,2024-07-24
id01,2024-08-26
id01,2024-08-28
id01,2024-08-29
id02,2024-08-23
id02,2024-09-24
id02,2024-09-26
id02,2024-09-27
id03,2024-08-30
id03,2024-09-01
id03,2024-09-02
id03,2024-09-06
id04,2024-09-03
id04,2024-10-10
id04,2024-10-12
id04,2024-10-13
id05,2024-10-19
id05,2024-10-23
id05,2024-10-24
id05,2024-10-27
id06,2024-07-25
id06,2024-07-26
id06,2024-07-27
id06,2024-07-30
id07,2024-07-07
id07,2024-08-02
id07,2024-08-04
id07,2024-08-05
id08,2024-08-28
id08,2024-08-29
id08,2024-08-30
id08,2024-09-02
id09,2024-08-02
id09,2024-08-31
id09,2024-09-02
id09,2024-09-03
id10,2024-08-28
id10,2024-08-30
id10,2024-08-31
id10,2024-09-03
"""

# DataFrame 생성
valid_ids = pd.read_csv(StringIO(string), sep=',')
valid_ids['pk'] = valid_ids['subject_id']+valid_ids['sleep_date']

### 📁 데이터 읽기

In [7]:
# [1]공통
train1 = pd.read_parquet(f'{PATH}/train_63775_vF.parquet')
test1 = pd.read_parquet(f'{PATH}/test_63775_vF.parquet')

# [2]version2 train,test 데이터셋 by 현종열
train2 = pd.read_parquet(f'{PATH}/train_hjy_0603_v1.parquet')
test2 = pd.read_parquet(f'{PATH}/test_hjy_0603_v1.parquet')

# [1]+[2]
a1 = train1.columns.tolist()
a2 = train2.columns.tolist()
feats = ['subject_id','sleep_date','lifelog_date']+list(set(a2)-set(a1))
train2 = train2[feats].copy()
test2 = test2[feats].copy()
train = train1.merge(train2,on=['subject_id','sleep_date','lifelog_date'],how='left')
test = test1.merge(test2,on=['subject_id','sleep_date','lifelog_date'],how='left')

# [3]QWEN3 8B 활용한 결측처리 (대상: mScreenStatus)
mScreenStatus_llm = pd.read_excel(f'{PATH}/mScreenStatus_llm결측값생성후파생변수생성_20250609_v1.xlsx')
feats = ['sleep_time', 'wake_time', 'sleep_duration_min', 'avg_sleep_time', 'avg_wake_time', 'avg_sleep_duration', 'sleep_time_diff', 'wake_time_diff', 'sleep_duration_diff', 'sleep_time_ratio', 'wake_time_ratio', 'sleep_duration_ratio', 'sleep_time_lag1', 'wake_time_lag1', 'sleep_duration_lag1', 'sleep_time_diff_lag1', 'wake_time_diff_lag1', 'sleep_duration_diff_lag1', 'sleep_time_ratio_lag1', 'wake_time_ratio_lag1', 'sleep_duration_ratio_lag1', 'sleep_time_lag2', 'wake_time_lag2', 'sleep_duration_lag2', 'sleep_time_diff_lag2', 'wake_time_diff_lag2', 'sleep_duration_diff_lag2', 'sleep_time_ratio_lag2', 'wake_time_ratio_lag2', 'sleep_duration_ratio_lag2', 'sleep_time_mean2d', 'wake_time_mean2d', 'sleep_duration_min_mean2d', 'sleep_time_diff_mean2d', 'wake_time_diff_mean2d', 'sleep_duration_diff_mean2d', 'sleep_time_ratio_mean2d', 'wake_time_ratio_mean2d', 'sleep_duration_ratio_mean2d', 'sleep_time_std2d', 'wake_time_std2d', 'sleep_duration_min_std2d', 'sleep_time_diff_std2d', 'wake_time_diff_std2d', 'sleep_duration_diff_std2d', 'sleep_time_ratio_std2d', 'wake_time_ratio_std2d', 'sleep_duration_ratio_std2d', 'sleep_time_mean3d', 'wake_time_mean3d', 'sleep_duration_min_mean3d', 'sleep_time_diff_mean3d', 'wake_time_diff_mean3d', 'sleep_duration_diff_mean3d', 'sleep_time_ratio_mean3d', 'wake_time_ratio_mean3d', 'sleep_duration_ratio_mean3d', 'sleep_time_std3d', 'wake_time_std3d', 'sleep_duration_min_std3d', 'sleep_time_diff_std3d', 'wake_time_diff_std3d', 'sleep_duration_diff_std3d', 'sleep_time_ratio_std3d', 'wake_time_ratio_std3d', 'sleep_duration_ratio_std3d', 'sleep_time_mean5d', 'wake_time_mean5d', 'sleep_duration_min_mean5d', 'sleep_time_diff_mean5d', 'wake_time_diff_mean5d', 'sleep_duration_diff_mean5d', 'sleep_time_ratio_mean5d', 'wake_time_ratio_mean5d', 'sleep_duration_ratio_mean5d', 'sleep_time_std5d', 'wake_time_std5d', 'sleep_duration_min_std5d', 'sleep_time_diff_std5d', 'wake_time_diff_std5d', 'sleep_duration_diff_std5d', 'sleep_time_ratio_std5d', 'wake_time_ratio_std5d', 'sleep_duration_ratio_std5d', 'sleep_time_mean7d', 'wake_time_mean7d', 'sleep_duration_min_mean7d', 'sleep_time_diff_mean7d', 'wake_time_diff_mean7d', 'sleep_duration_diff_mean7d', 'sleep_time_ratio_mean7d', 'wake_time_ratio_mean7d', 'sleep_duration_ratio_mean7d', 'sleep_time_std7d', 'wake_time_std7d', 'sleep_duration_min_std7d', 'sleep_time_diff_std7d', 'wake_time_diff_std7d', 'sleep_duration_diff_std7d', 'sleep_time_ratio_std7d', 'wake_time_ratio_std7d', 'sleep_duration_ratio_std7d', 'weekday_avg_sleep', 'sleep_duration_weekday_avg_diff', 'sleep_duration_weekday_avg_div']
drop_features = [i for i in feats if i in train.columns]
train = train.drop(columns=drop_features)
train = train.merge(mScreenStatus_llm,on=['subject_id','lifelog_date'],how='left')
test = test.drop(columns=drop_features)
test = test.merge(mScreenStatus_llm,on=['subject_id','lifelog_date'],how='left')

In [8]:
def sanitize_column_names(df):
    df.columns = df.columns.str.replace(r'[^\w]', '_', regex=True)
    return df

# train = sanitize_column_names(train)
# test = sanitize_column_names(test)

drop_featurs = [
 'Unnamed: 0'
,'light_week_type_lag1'
,'week_type'
,'week_type_lag1'
,'activehour_top_bssid'
,'beforebed_top_bssid'
]
drop_featurs = [i for i in drop_featurs if i in train.columns]
train = train.drop(columns=drop_featurs)
test = test.drop(columns=drop_featurs)

# check
# train  shape: (575, 357)
# test   shape: (250, 357)
print('# train  shape:',train.shape)
print('# test   shape:',test.shape)

# train  shape: (450, 357)
# test   shape: (250, 357)


In [9]:
# weekend
train['weekend'] = np.where(train['weekday'].isin(['토요일','금요일']),1,0)
test['weekend'] = np.where(test['weekday'].isin(['토요일','금요일']),1,0)

In [10]:
# 숫자형 컬럼만 선택해서 결측값 -1로 채우기
train[train.select_dtypes(include='number').columns] = train.select_dtypes(include='number').fillna(-1)
test[test.select_dtypes(include='number').columns] = test.select_dtypes(include='number').fillna(-1)

### 🔨 모델 학습

In [11]:
%%time

# CPU times: user 6h 43min 15s, sys: 2min 24s, total: 6h 45min 39s
# Wall time: 2h 17min 41s

common_params = {
  'n_estimators': 5000, # 5000
  "learning_rate": 0.01, # 0.01
  'lambda_l1': 5, # 5
  'lambda_l2': 1, # 1
  'bagging_fraction': 0.8, # 0.8
  'feature_fraction': 1,
  'n_jobs': -1,
  'verbosity': -1
}

xgb_params = {
    'n_estimators': 1000,
    'learning_rate': 0.01,
    'max_depth': 6,
    'min_child_weight': 1, # 1
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42
}

xgb_params_S1 = {
    'n_estimators': 1000,
    'learning_rate': 0.01,
    'max_depth': 6,
    'min_child_weight': 5, # 1
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42
}

tabpfn_params = {
    'device': 'cuda',  # GPU 사용
}

# 모델별 세부 하이퍼파라미터
best_param_dict = {}
best_param_dict['Q3'] = common_params
best_param_dict['S1'] = common_params
best_param_dict['S2'] = common_params
best_param_dict['S3'] = common_params
best_param_dict['Q1'] = common_params
best_param_dict['Q2'] = common_params

# drop_featurs
drop_featurs = [
 'light_month'
]
drop_featurs = [i for i in drop_featurs if i in train.columns]
train = train.drop(columns=drop_featurs)
test = test.drop(columns=drop_featurs)

# run_basemodel
submission_final, oof_result = run_basemodel(train, test, valid_ids, best_param_dict, n_splits=5, n_repeats=5, random_state=42, AutoTab=False, S1_balance=True)

weekday
# X shape: (450, 428)
# test_X shape: (250, 428)
# valid_ids shape: (40, 3)

 STEP1: 실험 결과 확인
=============== Validation Results ==============
# Q1 of xfeatures2: 170
# xfeatures2: ['Q1_weekend_te', 'Q1_weekend_te2', 'Q1_new6_te', 'Q1_new6_te2', 'Q1_new8_te', 'Q1_new8_te2', 'Q1_new7_te', 'Q1_new7_te2', 'Q1_new2_te', 'Q1_new1_te', 'Q1_new2_te2', 'Q1_new1_te2', 'wake_time_min', 'beforebed_통화&전화_앱이용시간', 'wake_time', 'sleep_charging_transitions', 'sleep_duration_vs_weekday_avg', 'S1_weekend_te', 'S1_weekend_te2', 'beforebed_통화_time', 'S3_new8_te', 'S3_new8_te2', 'wake_time_ratio', 'light_avg_wake_time', 'wake_time_max', 'sleep_duration_min_min', 'S3_weekend_te', 'S3_weekend_te2', 'S3_new1_te', 'S3_new1_te2', 'wake_time_ratio_mean3d', 'S3_new2_te', 'S3_new6_te', 'S3_new2_te2', 'wake_time_diff', 'S3_new6_te2', 'wake_time_ratio_mean5d', 'S3_new7_te', 'S3_new7_te2', 'wake_time_ratio_mean2d', 'light_rolling_wake_time_3d', 'wake_time_diff_mean5d', 'vacation', 'sleep_duration_ratio_mean5

,학습sum,학습len,학습mean,테스트sum,테스트len,테스트mean
Q1,223,450,0.4956,121,250,0.4840
Q2,253,450,0.5622,148,250,0.5920
Q3,270,450,0.6000,176,250,0.7040
S1,390,450,0.8667,185,250,0.7400
S2,293,450,0.6511,172,250,0.6880
S3,298,450,0.6622,167,250,0.6680


,proportion,proportion
S1,,
1,0.4978,0.5240
0,0.3178,0.3760
2,0.1844,0.1000


CPU times: user 5min 20s, sys: 2.09 s, total: 5min 22s
Wall time: 3min 28s


### 🔨 최종 제출파일

In [12]:
"""
weekday
# X shape: (450, 428)
# test_X shape: (250, 428)
# valid_ids shape: (40, 3)

 STEP1: 실험 결과 확인
=============== Validation Results ==============
# Q1 of xfeatures2: 170
# xfeatures2: ['Q1_weekend_te', 'Q1_weekend_te2', 'Q1_new6_te', 'Q1_new6_te2', 'Q1_new8_te', 'Q1_new8_te2', 'Q1_new7_te', 'Q1_new7_te2', 'Q1_new2_te', 'Q1_new1_te', 'Q1_new2_te2', 'Q1_new1_te2', 'wake_time_min', 'beforebed_통화&전화_앱이용시간', 'wake_time', 'sleep_charging_transitions', 'sleep_duration_vs_weekday_avg', 'S1_weekend_te', 'S1_weekend_te2', 'beforebed_통화_time', 'S3_new8_te', 'S3_new8_te2', 'wake_time_ratio', 'light_avg_wake_time', 'wake_time_max', 'sleep_duration_min_min', 'S3_weekend_te', 'S3_weekend_te2', 'S3_new1_te', 'S3_new1_te2', 'wake_time_ratio_mean3d', 'S3_new2_te', 'S3_new6_te', 'S3_new2_te2', 'wake_time_diff', 'S3_new6_te2', 'wake_time_ratio_mean5d', 'S3_new7_te', 'S3_new7_te2', 'wake_time_ratio_mean2d', 'light_rolling_wake_time_3d', 'wake_time_diff_mean5d', 'vacation', 'sleep_duration_ratio_mean5d', 'wake_time_diff_mean3d', 'Q3_new7_te2', 'Q3_new7_te', 'img1', 'wake_time_diff_mean2d', 'month', 'sleep_duration_min', 'new2', 'light_avg_sleep_time', 'sleep_duration_diff_mean5d', 'light_night_mean', 'light_awake_blocks', 'beforebed_전화_time', 'light_avg_sleep_duration', 'sleep_duration_min_mean2d', 'wake_time_mean2d', 'm_activity_0@240min@std@08h00m', 'light_rolling_sleep_duration_3d', 'light_weekday_avg_sleep', 'new6', 'sleep_time_ratio_std3d', 'new8', 'work_hour_rssi_max', 'sleep_duration_ratio_mean3d', 'sleep_duration_min_max', 'work_hour_others_ratio', 'sleep_duration_weekday_avg_div', 'm_light_awake_blocks', 'm_activity_0@240min@sum@12h00m', 'wake_time_ratio_mean7d', 'wake_time_diff_mean7d', 'light_wake_time_diff', 'Q3_weekend_te2', 'sleep_duration_ratio_mean2d', 'light_sleep_duration_vs_weekday_avg', 'Q3_weekend_te', 'Q3_new6_te2', 'Q3_new6_te', 'avg_sleep_duration', 'wake_time_ratio_lag2', 'light_rolling_sleep_duration_2d', 'sleep_time_ratio_std5d', 'S1_new6_te', 'S1_new6_te2', 'free_hour_others_ratio', 'sleep_duration_ratio_lag1', 'new7', 'Q2_weekend_te2', 'sleep_time_ratio_std7d', 'sleep_duration_weekday_avg_diff', 'Q2_weekend_te', 'm_activity_0@240min@sum@08h00m', 'w_light_awake_blocks', 'sleep_duration_ratio_lag2', 'wake_time_ratio_std7d', 'Q3_new2_te2', 'sleep_duration_min_mean3d', 'weekend2', 'weekend', 'sleep_duration_ratio_mean7d', 'Q3_new2_te', 'Q2_new7_te2', 'avg_wake_time', 'Q2_new7_te', 'wake_time_ratio_lag1', 'activehour_NAVER_time', 'sleep_duration_diff_mean3d', 'wake_time_diff_std7d', '공휴일', 'light_sleep_duration_diff', 'img4', 'sleep_duration_min_mean5d', 'wake_time_ratio_std5d', 'sleep_duration_ratio', 'light_rolling_wake_time_2d', 'm_activity_0@240min@std@00h00m', 'vehicle_minutes', 'sleep_duration_diff_mean2d', 'sleep_time_ratio_mean5d', 'light_sleep_time_diff', 'light_sleep_duration_lag1', 'wake_time_ratio_std3d', 'sleep_time_ratio_mean3d', 'm_light_awake_minutes', 'sleep_duration_diff_mean7d', 'sleep_duration_lag1', 'light_sleep_duration_lag2', 'Q3_new8_te2', 'S1_new2_te', 'wake_time_std7d', 'S1_new2_te2', 'Q2_new1_te2', 'activehour_hour_span_minutes', 'Q3_new8_te', 'activehour_메신저_time', 'Q2_new1_te', 'wake_time_mean3d', 'light_wake_time_lag2', 'sleep_time_ratio_lag2', 'activehour_avg_rssi', 'sleep_duration_diff', 'sleep_time_ratio_lag1', 'sleep_time_ratio_mean7d', 'sleep_vehicle_minutes', 'active_hour_vehicle_minutes', 'sleep_time_ratio_mean2d', 'm_activity_0@240min@std@12h00m', 'weekday_avg_sleep', 'sleep_time_diff_std5d', 'Q2_new2_te2', 'wake_time_diff_std5d', 'Q2_new2_te', 'sleep_time_diff_std7d', 'work_hour_rssi_min', 'S1_new8_te2', 'sleep_time_lag2', 'S1_new8_te', 'work_hour_rssi_mean', 'm_activity@240min@std@08h00m', 'sleep_hour_walk_minutes', 'sleep_activity_minutes', 'light_sleep_duration_min', 'sleep_time_diff_std3d', 'wake_time_ratio_std2d', 'sleep_time_ratio_std2d', 'wake_time_lag1']
=> Q1 of xfeatures: 170

# Q2 of xfeatures2: 83
# xfeatures2: ['Q2_new7_te2', 'Q2_new7_te', 'Q2_new2_te2', 'Q2_new2_te', 'Q2_new6_te2', 'Q2_new6_te', 'Q2_new8_te2', 'Q2_new8_te', 'Q2_new1_te2', 'Q2_new1_te', 'Q2_weekend_te2', 'Q2_weekend_te', 'Q3_new6_te2', 'Q3_new6_te', 'Q3_new2_te2', 'Q3_new2_te', 'Q3_new8_te2', 'Q3_new1_te2', 'Q3_new7_te2', 'Q3_weekend_te2', 'Q3_new8_te', 'Q3_new1_te', 'Q3_new7_te', 'Q3_weekend_te', 'activehour_unique_bssid_count', 'beforebed_unique_bssid_count', 'm_activity@240min@std@04h00m', 'light_day_mean', 'm_activity_0@240min@sum@20h00m', 'light_mean', 'light_std', 'Q1_new7_te2', 'activity_minutes', 'Q1_new7_te', 'm_activity_0@240min@sum@04h00m', 'light_max', 'walking_minutes', 'm_activity@240min@std@20h00m', 'm_activity_0@240min@std@20h00m', '불끈시간부터기상시간', 'sleep_time_diff_mean5d', 'sleep_time_diff_mean7d', 'sleep_time_mean2d', 'Q1_weekend_te2', 'sleep_time_mean7d', 'sleep_time_mean5d', 'charging_transitions', 'lights_off_time', 'activehour_메신저_time', 'charging_sum', 'sleep_time', 'Q1_new1_te2', 'Q1_weekend_te', 'activehour_scan_count', 'sleep_time_mean3d', 'sleep_hour_mean_speed', 'wake_time_mean2d', 'beforebed_hour_span_minutes', 'Q1_new1_te', 'Q1_new2_te2', 'm_activity_met@240min@sum@12h00m', 'wake_time_mean3d', 'Q1_new2_te', 'beforebed_min_rssi', 'charging_ratio', 'beforebed_avg_rssi', 'sleep_time_ratio_mean5d', 'sleep_time_diff_mean3d', 'm_activity_met@240min@sum@08h00m', 'vehicle_minutes', 'sleep_time_ratio_mean7d', 'free_hour_unknown_ratio', 'activehour_screen_time_vs_avg_pct', 'sleep_time_min', 'beforebed_max_rssi', 'sleep_time_diff_lag2', 'm_activity_met@240min@std@04h00m', 'sleep_time_diff_mean2d', 'activehour_OneUI홈_time', 'wake_time_diff_lag1', 'vacation', 'm_activity_0@240min@std@16h00m', 'rolling_wake_time_3d']
=> Q2 of xfeatures: 83

# Q3 of xfeatures1: 15
# xfeatures1: ['Q3_new2_te', 'Q3_weekend_te2', 'Q3_weekend_te', 'walking_minutes', 'Q3_new1_te', 'beforebed_scan_count', 'Q3_new8_te2', 'Q3_new1_te2', 'Q3_new2_te2', 'sleep_duration_ratio_mean3d', 'Q3_new6_te2', 'Q3_new7_te2', 'Q3_new7_te', 'Q3_new6_te', 'Q3_new8_te']
# Q3 of xfeatures2: 88
# xfeatures2: ['Q3_weekend_te2', 'Q3_weekend_te', 'Q3_new8_te2', 'Q3_new8_te', 'Q3_new2_te2', 'Q3_new2_te', 'Q3_new1_te2', 'Q3_new6_te2', 'Q3_new1_te', 'Q3_new6_te', 'Q3_new7_te2', 'Q3_new7_te', 'beforebed_scan_count', 'beforebed_top_bssid_count', 'light_weekday_avg_sleep', 'work_hour_others_ratio', 'light_avg_sleep_duration', 'Q2_new6_te2', 'free_hour_others_ratio', 'Q2_new6_te', 'Q2_weekend_te2', 'Q2_new2_te2', 'Q2_weekend_te', 'Q2_new2_te', 'Q2_new8_te2', 'Q2_new1_te2', 'Q2_new8_te', 'month', 'Q2_new1_te', 'light_avg_sleep_time', 'beforebed_hour_span_minutes', 'Q1_new7_te2', 'Q1_new7_te', 'Q2_new7_te2', 'Q2_new7_te', 'sleep_time_m_light_sleep_time', 'sleep_hour_others_ratio', 'light_avg_wake_time', 'weekend3', 'beforebed_strong_signal_ratio', 'beforebed_empty_scan_count', 'work_hour_rssi_min', 'light_weekday', 'work_hour_rssi_max', 'activehour_hour_span_minutes', 'work_hour_rssi_mean', 'm_activity_0@240min@std@12h00m', 'light_sleep_time_diff', 'Q1_new6_te2', 'active_hour_vehicle_minutes', 'm_activity_0@240min@sum@12h00m', 'light_mean', 'Q1_new6_te', 'Q1_new2_te2', 'sleep_hour_max_light', 'weekend_holilday', 'Q1_weekend_te2', 'Q1_new2_te', 'light_rolling_sleep_duration_3d', 'mgps_first_wakeup_minutes', 'Q1_weekend_te', 'activehour_NAVER_time', 'light_rolling_sleep_time_3d', 'light_day_mean', 'S3_weekend_te', 'light_max', 'light_rolling_wake_time_3d', 'beforebed_min_rssi', 'S3_weekend_te2', 'beforebed_avg_rssi', 'sleep_hour_min_speed', 'sleep_duration_min_m_light_sleep_duration_min', 'Q1_new8_te2', 'S1_new2_te2', 'sleep_duration_min_std5d', 'sleep_duration_min_std3d', 'light_std', 'S1_new2_te', 'Q1_new8_te', 'light_sleep_duration_lag2', 'light_night_mean', 'm_activity_met@240min@sum@04h00m', 'light_rolling_sleep_duration_2d', 'weekend2', 'weekend', 'light_sleep_duration_lag1', 'sleep_time_min_mLight', 'hour_slept_mLight']
=> Q3 of xfeatures: 90

# S2 of xfeatures2: 104
# xfeatures2: ['S2_new1_te', 'S2_new7_te', 'S2_new1_te2', 'S2_new7_te2', 'S2_new2_te', 'S2_new6_te', 'S2_new2_te2', 'S2_new6_te2', 'S2_new8_te', 'S2_new8_te2', 'S2_weekend_te', 'S2_weekend_te2', 'S3_new2_te', 'S3_new2_te2', 'S3_new7_te', 'S3_new7_te2', 'S3_new1_te', 'S3_new1_te2', 'S3_new6_te2', 'S3_new6_te', 'S1_new2_te', 'S1_new2_te2', 'S3_weekend_te', 'S3_weekend_te2', 'S1_new7_te', 'S1_new7_te2', 'S1_new6_te', 'S1_new6_te2', 'S3_new8_te', 'S3_new8_te2', 'S1_new8_te', 'S1_new8_te2', 'S1_new1_te', 'S1_new1_te2', 'S1_weekend_te', 'S1_weekend_te2', 'beforebed_캐시워크_time', 'activehour_캐시워크_time', 'month', 'beforebed_max_rssi', 'activehour_max_rssi', 'activehour_전화_time', 'avg_sleep_time', 'sleep_time_diff', 'sleep_time_diff_mean7d', 'light_wake_time_lag2', 'active_hour_max_speed', 'light_rolling_sleep_duration_2d', 'light_sleep_time_lag1', 'activehour_unique_app_count', 'activehour_scan_count', 'sleep_time_diff_mean5d', 'light_sleep_duration_lag1', 'sleep_duration_vs_weekday_avg', 'beforebed_통화_time', 'beforebed_통화&전화_앱이용시간', 'sleep_time_diff_mean3d', 'light_sleep_duration_lag2', 'sleep_time_diff_mean2d', 'light_rolling_sleep_duration_3d', 'rolling_sleep_duration_2d', 'light_sleep_time_lag2', 'light_rolling_wake_time_2d', 'img1', 'wake_time_ratio_lag2', 'm_activity_met@240min@std@12h00m', 'rolling_sleep_duration_3d', 'sleep_time_ratio_mean7d', 'sleep_duration_ratio_lag2', 'sleep_duration_min_max', 'm_activity@240min@std@00h00m', 'm_activity_met@240min@std@00h00m', 'sleep_time_lag2', 'sleep_time_diff_std3d', 'work_hour_unknown_ratio', 'light_sleep_time_diff_lag1', 'm_activity_0@240min@sum@20h00m', 'light_rolling_wake_time_3d', 'new6', 'sleep_time_diff_std5d', 'rolling_sleep_time_3d', 'm_activity@240min@std@12h00m', 'free_hour_unknown_ratio', 'sleep_time_ratio', 'light_sleep_duration_min', 'Q1_new7_te2', 'Q1_new7_te', 'light_night_ratio', 'sleep_max_charging_duration', 'img3', 'active_hour_distance_x', 'sleep_time_ratio_mean3d', 'sleep_avg_charging_duration', 'light_rolling_sleep_time_2d', 'sleep_charging_ratio', 'm_activity_0@240min@std@12h00m', 'activehour_top_bssid_count', 'sleep_charging_sum', 'is_sleep_duration_change_large', 'm_activity_0@240min@std@00h00m', 'sleep_time_diff_lag2', 'avg_charging_duration', 'active_hour_mean_speed', 'sleep_time_diff_lag1']
=> S2 of xfeatures: 104

# S3 of xfeatures2: 96
# xfeatures2: ['S3_weekend_te', 'S3_weekend_te2', 'S3_new8_te', 'S3_new2_te', 'S3_new2_te2', 'S3_new8_te2', 'S3_new7_te', 'S3_new6_te', 'S3_new6_te2', 'S3_new7_te2', 'S3_new1_te', 'S3_new1_te2', 'S2_new2_te', 'S2_new2_te2', 'S2_new7_te', 'S2_new7_te2', 'S2_weekend_te', 'S2_weekend_te2', 'S2_new1_te', 'S2_new6_te', 'S2_new1_te2', 'S2_new6_te2', 'S2_new8_te', 'S2_new8_te2', 'month', 'activehour_캐시워크_time', 'Q1_new8_te2', 'Q1_new8_te', 'Q1_new2_te2', 'Q1_new2_te', 'Q1_new1_te2', 'Q1_new1_te', 'Q1_new7_te2', 'Q1_new6_te2', 'Q1_new7_te', 'Q1_new6_te', 'Q1_weekend_te2', 'activehour_max_rssi', 'Q1_weekend_te', 'beforebed_캐시워크_time', 'S1_new2_te', 'S1_new2_te2', 'S1_new7_te', 'work_hour_rssi_max', 'S1_new7_te2', 'S1_new6_te', 'S1_new6_te2', 'beforebed_통화_time', 'beforebed_통화&전화_앱이용시간', 'S1_new1_te', 'Q3_weekend_te2', 'S1_new1_te2', 'Q3_weekend_te', 'S1_new8_te', 'avg_wake_time', 'beforebed_max_rssi', 'S1_new8_te2', 'wake_time_min', 'activehour_unique_app_count', 'Q3_new7_te2', 'Q3_new2_te2', 'Q3_new7_te', 'S1_weekend_te', 'Q3_new2_te', 'active_hour_high_hr', 'S1_weekend_te2', 'Q3_new6_te2', 'Q3_new6_te', 'active_hour_walk_minutes', 'activehour_성경일독Q_time', 'sleep_duration_min', 'avg_sleep_duration', 'sleep_duration_min_min', 'img3', 'beforebed_성경일독Q_time', 'Q3_new1_te2', 'm_activity_0@240min@std@20h00m', 'active_hour_max_speed', 'Q3_new1_te', 'wake_time', 'wake_time_mean7d', 'sleep_duration_min_mean3d', 'activehour_전화_time', 'activehour_통화_time', 'beforebed_NAVER_time', 'm_activity_0@240min@sum@04h00m', 'new8', 'img4', 'Q3_new8_te2', 'm_activity_0@240min@std@04h00m', 'wake_time_mean3d', 'weekday_avg_sleep', 'Q3_new8_te', 'active_hour_distance_x', 'active_hour_jog_minutes', 'm_activity@240min@std@12h00m']
=> S3 of xfeatures: 96

# S1 of xfeatures2: 173
# xfeatures2: ['S1_weekend_te2', 'S1_weekend_te', 'S1_new8_te', 'S1_new8_te2', 'S1_new7_te', 'S1_new7_te2', 'S1_new2_te', 'S1_new2_te2', 'S1_new6_te', 'S1_new6_te2', 'S1_new1_te', 'S1_new1_te2', 'S2_new8_te', 'S2_new8_te2', 'S2_new2_te', 'S2_new2_te2', 'sleep_duration_min_max', 'S2_new7_te', 'S2_new7_te2', 'S2_new6_te', 'S2_new6_te2', 'S2_weekend_te', 'S2_weekend_te2', 'sleep_duration_min', 'wake_time_max', 'rolling_sleep_duration_2d', 'S2_new1_te', 'S2_new1_te2', 'avg_sleep_duration', 'sleep_duration_min_mean2d', 'rolling_sleep_duration_3d', 'weekday_avg_sleep', 'sleep_duration_vs_weekday_avg', 'wake_time', 'Q1_weekend_te', 'Q1_weekend_te2', 'vacation', 'new6', 'predweekday', 'sleep_duration_min_mean3d', 'sleep_duration_min_mean5d', 'sleep_duration_min_mean7d', 'light_rolling_sleep_duration_3d', 'light_rolling_sleep_duration_2d', 'weekend', 'weekend2', 'light_sleep_duration_min', 'new8', 'img1', 'm_light_awake_minutes', 'S3_new2_te2', 'S3_new2_te', 'sleep_duration_weekday_avg_div', 'new3', 'sleep_avg_charging_duration', 'sleep_time_diff', 'sleep_time_d_light_sleep_time', 'S3_new7_te2', 'S3_new7_te', 'activehour_unique_app_count', 'sleep_time_diff_mean2d', 'sleep_duration_weekday_avg_diff', 'sleep_max_charging_duration', 'avg_sleep_time', 'beforebed_핸드폰사용시간', 'beforebed_total_screen_time', 'sleep_duration_diff', 'avg_wake_time', 'sleep_duration_ratio', 'light_weekday_avg_sleep', 'activehour_top_bssid_count', 'wake_time_ratio', 'sleep_time_diff_mean3d', 'wake_time_diff', 'sleep_time_diff_mean7d', 'light_rolling_wake_time_3d', 'light_sleep_duration_lag2', 'light_avg_sleep_duration', 'sleep_time_min', 'sleep_duration_min_min', 'S3_new6_te2', 'light_sleep_time', 'S3_new6_te', 'light_night_mean', 'activehour_scan_count', 'activehour_hour_span_minutes', 'sleep_time_diff_lag1', 'sleep_time_mean2d', 'wake_time_mean2d', 'sleep_time_diff_mean5d', 'light_is_sleep_duration_change_large', 'light_wake_time_lag2', 'sleep_duration_ratio_mean2d', 'Q1_new6_te', 'sleep_charging_ratio', 'S3_new1_te2', 'Q1_new6_te2', 'S3_new1_te', 'sleep_charging_sum', 'beforebed_unique_app_count', 'light_sleep_duration_lag1', 'sleep_time', 'S3_new8_te', 'S3_new8_te2', 'light_sleep_duration_ratio', 'wake_time_min', 'sleep_duration_diff_mean2d', 'sleep_time_mean7d', 'sleep_duration_ratio_mean7d', 'sleep_duration_min_d_light_sleep_duration_min', 'light_avg_wake_time', 'sleep_duration_diff_mean7d', 'wake_time_ratio_mean5d', 'work_hour_others_ratio', 'wake_time_ratio_mean3d', 'S3_weekend_te2', 'S3_weekend_te', 'activehour_total_screen_time', 'sleep_time_mean5d', 'Q1_new2_te', 'sleep_time_mean3d', 'wake_time_ratio_mean2d', 'Q1_new2_te2', 'Q3_new2_te', 'Q3_new2_te2', 'beforebed_max_rssi', 'new5', 'sleep_duration_interp_mLight', 'sleep_duration_min_mLight', 'wake_time_ratio_lag2', 'activehour_max_rssi', 'light_rolling_wake_time_2d', 'wake_time_diff_mean5d', 'sleep_duration_ratio_mean5d', 'beforebed_scan_count', 'wake_time_diff_mean3d', 'light_awake_blocks', 'new7', 'sleep_duration_diff_mean5d', 'sleep_duration_ratio_mean3d', 'Q1_new8_te', 'weekend3', 'Q1_new8_te2', 'is_sleep_duration_change_large', 'sleep_duration_lag1', 'light_sleep_duration_vs_weekday_avg', 'light_awake_minutes', 'wake_time_ratio_mean7d', 'sleep_duration_interp_hour_mLight', 'sleep_duration_hour_mLight', 'wake_time_diff_mean2d', 'activehour_min_rssi', 'Q3_new7_te', 'Q3_new7_te2', 'active_hour_min_speed', 'activehour_NAVER_time', 'wake_time_diff_mean7d', 'beforebed_OneUI홈_time', 'sleep_duration_diff_mean3d', 'sleep_time_diff_lag2', 'sleep_time_max', 'img2', 'sleep_hour_rssi_min', 'w_light_awake_minutes', 'img3', 'wake_time_mean3d', 'work_hour_rssi_mean', 'light_sleep_time_lag2', 'light_wake_time', 'beforebed_hour_span_minutes', 'beforebed_캐시워크_time', 'activehour_empty_scan_count', 'rolling_wake_time_3d']
=> S1 of xfeatures: 173

 평균 F1: 0.7128 / [상세] Q1(기상직후수면질):0.7206 Q2(취침전신체적피로):0.8400 Q3(취침전스트레스):0.7151 S2(수면효율):0.7749 S3(수면잠들기시간):0.7234 S1(S1):0.5025
# 전체 평균 F1: 0.7128
================================================

 STEP2: 전체 데이터로 모델 재학습
====== modeling with 100% train & no valid =====
# submission_0.7127633424733214.csv 저장 완료
# submission shape:(250, 9)
================================================

Top 10 Weight Combinations:
Rank 1: lgb_A=0.5, xgb_B=0.2, tab_C=0.3 - Score: 0.7218
Saved submission to submission_top1_0.7218.csv
Rank 2: lgb_A=0.4, xgb_B=0.1, tab_C=0.5 - Score: 0.7202
Saved submission to submission_top2_0.7202.csv
Rank 3: lgb_A=0.5, xgb_B=0.3, tab_C=0.2 - Score: 0.7176
Saved submission to submission_top3_0.7176.csv
Rank 4: lgb_A=0.3, xgb_B=0.2, tab_C=0.5 - Score: 0.7167
Saved submission to submission_top4_0.7167.csv
Rank 5: lgb_A=0.5, xgb_B=0.1, tab_C=0.4 - Score: 0.7163
Saved submission to submission_top5_0.7163.csv
Rank 6: lgb_A=0.4, xgb_B=0.3, tab_C=0.3 - Score: 0.7163
Saved submission to submission_top6_0.7163.csv
Rank 7: lgb_A=0.4, xgb_B=0.2, tab_C=0.4 - Score: 0.7163
Saved submission to submission_top7_0.7163.csv
Rank 8: lgb_A=0.3, xgb_B=0.1, tab_C=0.6 - Score: 0.7158
Saved submission to submission_top8_0.7158.csv
Rank 9: lgb_A=0.5, xgb_B=0.0, tab_C=0.5 - Score: 0.7157
Saved submission to submission_top9_0.7157.csv
Rank 10: lgb_A=0.0, xgb_B=0.6, tab_C=0.4 - Score: 0.7152
Saved submission to submission_top10_0.7152.csv

 STEP3: 예측결과 비교표
학습sum	학습len	학습mean	테스트sum	테스트len	테스트mean
Q1	223	450	0.4956	121	250	0.4840
Q2	253	450	0.5622	148	250	0.5920
Q3	270	450	0.6000	176	250	0.7040
S1	390	450	0.8667	185	250	0.7400
S2	293	450	0.6511	172	250	0.6880
S3	298	450	0.6622	167	250	0.6680


proportion	proportion
S1
1	0.4978	0.5240
0	0.3178	0.3760
2	0.1844	0.1000

CPU times: user 5min 20s, sys: 2.09 s, total: 5min 22s
Wall time: 3min 28s
"""
from google.colab import files
files.download("submission_top1_0.7218.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>